# OpenAI Academy

[![Static Badge](https://img.shields.io/badge/Run%20in%20Modal%20-%20Agents?style=for-the-badge&color=green)](https://modal.com/notebooks/new/https://github.com/CelestoAI/agentor/blob/main/examples/openai-academy.ipynb)


## Agents with examples

In [ ]:
# %uv pip install openai-agents==0.3.3 agentor

## What is an Agent?

<img src="https://i.imgflip.com/a7rr5a.jpg" width="250px">

<br>

## Agents are LLMs with higher autonomy

- Tool access
- Memory

In [28]:
from dotenv import load_dotenv
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="Write a haiku about recursion in programming."
)

print(response.output[-1].content[0].text)

Recursive code hums
calling itself to finish
and echoes again

In [31]:
import datetime as dt

from agents import Agent, Runner

load_dotenv()

agent = Agent(name="Assistant", instructions="You are a helpful assistant", model="gpt-5-nano")

result = await Runner.run(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

Call itself again
until the base case returns
stack pops, calm again

In [3]:
agent = Agent(name="Assistant", instructions="Reply very concisely.")

result = await Runner.run(agent, "What city is the Golden Gate Bridge in?")
print(result.final_output)

San Francisco.


# Tool use

In [25]:
result = await Runner.run(agent, "What is the weather in London?")
print(result.final_output)

I don’t have live data. Please check a weather website or app for London’s current weather.

In [27]:
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

In [33]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-5-nano",
    tools=[get_weather],
)

In [34]:
result = await Runner.run(agent, "What is the weather in London?")
print(result.final_output)

Sunny London dawn
Quiet streets glow with warm light
London breathes summer

In [45]:
weather_tool_def = {
    "type": "function",
    "name": "get_weather",
    "description": "Retrieves current weather for the given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. London, United Kingdom"
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Units the temperature will be returned in."
            }
        },
        "required": ["location", "units"],
        "additionalProperties": False
    },
    "strict": True
}

def get_weather(location: str, units: str) -> str:
    """returns weather info for the specified location and units."""
    return f"The weather in {location} is sunny with a temperature of 20 degrees {units}."

tool_registry = {
    "get_weather": get_weather
}


In [41]:
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="What is the weather in London?",
    tools=[weather_tool_def]
)


In [48]:
print(response.output[-1])

ResponseFunctionToolCall(
    arguments='{"location":"London, United Kingdom","units":"celsius"}',
    call_id='call_nrQRnBrdprZfLraHuxz2K6rS',
    name='get_weather',
    type='function_call',
    id='fc_0cfc4a178f9c5f750068e2d72791148196a74f509b29b01236',
    status='completed'
)

In [49]:
import json

if response.output[-1].type == "function_call":
    fn_name = response.output[-1].name
    arguments = json.loads(response.output[-1].arguments)
    result = tool_registry[fn_name](**arguments)
    print(result)
else:
    print(response.output[-1].content[0].text)

The weather in London, United Kingdom is sunny with a temperature of 20 degrees celsius.

# Scheduling Agent

We will build an Agent with access to email and calendar to help schedule meetings on behalf of the user.

**Example conversation:**
- Lock wants a meeting
- Smith proposes a call
- AI assistant handles scheduling automatically

## Gmail and Calendar API

In [4]:
from agentor.integrations.google import GmailService, CalendarService, load_user_credentials
from rich import print

# Load your saved credentials
creds = load_user_credentials("credentials.my_google_account.json")

gmail = GmailService(creds)
calendar = CalendarService(creds)

In [ ]:
print("List emails", gmail.list_messages(limit=2))
print("Get an email", gmail.get_message(message_id="199a56838f299758"))
print("Get full email body", gmail.get_message_body(message_id="199a56838f299758"))

List emails
{
    'messages': [
        {'id': '199b58b30f5a36a7', 'threadId': '199b58b30f5a36a7'},
        {'id': '199b565a0687256a', 'threadId': '199b565a0687256a'}
    ],
    'nextPageToken': '02678382694700749137'
}

Get an email
{
    'message': {
        'id': '199a56838f299758',
        'threadId': '199a5672da2b36c4',
        'subject': 'Register for OpenAI Academy livestream',
        'from': 'An Moriarty <moriartyany@gmail.com>',
        'to': 'Aniket Maurya <theaniketmaurya@gmail.com>',
        'cc': '',
        'date': 'Thu, 2 Oct 2025 15:51:34 +0100',
        'snippet': 'Hi Aniket, Register for OpenAI Academy livestream to learn about Agentic systems. It&#39;s 
online and free! Thanks, Aniket',
        'labels': ['SENT']
    }
}

In [9]:
print("Search emails", gmail.search_messages(query="from:aniket"))

Search emails
{
    'messages': [
        {
            'id': '199a72a5b1e277b4',
            'threadId': '199a72a5b1e277b4',
            'subject': 'Declined: Weekly sync @ Mon Oct 6, 2025 4pm - 5pm (BST) (moriartyany@gmail.com)',
            'from': 'Aniket Maurya <theaniketmaurya@gmail.com>',
            'to': 'moriartyany@gmail.com',
            'cc': '',
            'date': 'Thu, 02 Oct 2025 23:03:14 +0000',
            'snippet': 'Weekly sync Aniket Maurya has declined this invitation. Process updates and blockers When 
Monday Oct 6, 2025 ⋅ 4pm – 5pm (United Kingdom Time) Organizer moriartyany@gmail.com Guests Aniket Maurya View',
            'labels': ['UNREAD', 'CATEGORY_PERSONAL', 'INBOX']
        },
        {
            'id': '199a72a485fac91c',
            'threadId': '199a72a485fac91c',
            'subject': 'Accepted: Weekly sync @ Mon Oct 6, 2025 4pm - 5pm (BST) (moriartyany@gmail.com)',
            'from': 'Aniket Maurya <theaniketmaurya@gmail.com>',
            'to': 'moriartyany@gmail.com',
            'cc': '',
            'date': 'Thu, 02 Oct 2025 23:03:09 +0000',
            'snippet': 'Weekly sync Aniket Maurya has accepted this invitation. Process updates and blockers When 
Monday Oct 6, 2025 ⋅ 4pm – 5pm (United Kingdom Time) Organizer moriartyany@gmail.com Guests Aniket Maurya View',
            'labels': ['UNREAD', 'CATEGORY_PERSONAL', 'INBOX']
        },
        {
            'id': '199740379d7befd6',
            'threadId': '199740379d7befd6',
            'subject': 'Welcome to Celesto AI',
            'from': 'aniket@celesto.ai',
            'to': 'moriartyany@gmail.com',
            'cc': '',
            'date': 'Tue, 23 Sep 2025 00:40:08 +0000',
            'snippet': 'Welcome to Celesto AI! Hi An Moriarty, Thanks for signing up for Celesto AI — we&#39;re 
excited to have you on board! Celesto AI helps you work smarter with complex documents — whether it&#39;s',
            'labels': ['CATEGORY_UPDATES', 'INBOX']
        }
    ]
}

In [ ]:
print("List calendar events", calendar.list_events())

List calendar events
{
    'events': [
        {
            'kind': 'calendar#event',
            'etag': '"3519250951695422"',
            'id': 'js4l888sfn1if2fug4encmusb4',
            'status': 'confirmed',
            'htmlLink': 
'https://www.google.com/calendar/event?eid=anM0bDg4OHNmbjFpZjJmdWc0ZW5jbXVzYjQgbW9yaWFydHlhbnlAbQ',
            'created': '2025-10-05T00:51:15.000Z',
            'updated': '2025-10-05T00:51:15.847Z',
            'summary': 'Celesto Test',
            'description': 'Process updates and blockers',
            'creator': {'email': 'moriartyany@gmail.com', 'self': True},
            'organizer': {'email': 'moriartyany@gmail.com', 'self': True},
            'start': {'dateTime': '2025-10-06T16:00:00+01:00', 'timeZone': 'Europe/London'},
            'end': {'dateTime': '2025-10-06T17:00:00+01:00', 'timeZone': 'Europe/London'},
            'iCalUID': 'js4l888sfn1if2fug4encmusb4@google.com',
            'sequence': 0,
            'attendees': [{'email': 'theaniketmaurya@gmail.com', 'responseStatus': 'needsAction'}],
            'reminders': {'useDefault': True},
            'eventType': 'default'
        }
    ],
    'nextPageToken': None
}

In [ ]:
event = {
    "summary": "OpenAI Academy - Agents SDK Session with Aniket",
    "description": "I attended the OpenAI Academy live session on Agents SDK with Aniket",
    "start": dt.datetime(2025, 10, 6, 15, 0, tzinfo=dt.timezone.utc),
    "end": dt.datetime(2025, 10, 6, 16, 0, tzinfo=dt.timezone.utc),
    "attendees": [{"email": "aniket@example.com"}],
}
calendar.create_event(event=event)

# Multi-agent system

## Agent handoff

## Agent as tool